### Business Problem 

Create a product that builds a brochure for a company that can be used for prospective clients, investors \
and potential recruits

Input - Will be company name and website 

In [5]:
## Importing libraries 

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [13]:
### Initialize and constants 

## when you call load_dotenv(), it searches for a .env file (by default in the same directory as your script), reads key-value pairs from that file
load_dotenv()
api_key = os.getenv('OPEN_API_KEY')


if api_key and api_key[:8] == 'sk-proj-':
    print("API key looks good to me")
else:
    print("There might be a problem with API key")

MODEL = 'gpt-4o-mini'
openai = OpenAI(api_key=api_key)


API key looks good to me


In [20]:
class Website:
    """
    Class that represent website that is scraped with links
    """
    url:str
    title:str
    body:str
    links: List[str]
    text:str


    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    
    

In [22]:
latitude = Website("https://www.latitudefinancial.com.au/")

print(latitude.links)

['/credit-cards/', '/personal-loan', 'https://loans.portal.latitudefinancial.com.au/s/quote', 'https://www.latitudefinancial.com.au/credit-cards/28-degrees/', 'https://loans.portal.latitudefinancial.com.au/s/quote', 'https://www.latitudefinancial.com.au/credit-cards/28-degrees/', '/personal-loan-repayment-calculator/', '/budget-planner/', 'https://www.latitudefinancial.com.au/interest-free/find-a-store/', '/car-loan/', '/debt-consolidation-loans/', '/home-renovation-loans/', '/credit-cards/', 'https://latitudefs.zendesk.com/hc/en-au', 'https://www.creditsmart.org.au/', '/hardship-care/', '/digital-wallets/apple-pay/', '/vulnerable-customers/']


## Ask gpt4-o-mini to figure out the links which are relevant

#### As there are lots of links, it difficult to go through each one of them and find which link can contains the details we need for a brochure and some of the links can be just be a waste. 

In [25]:
## This is an example of oneshot prompting, because we have provided one example of how the output should look

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About Page, or a Company page, or Careers/Jobs pages. \n"

link_system_prompt += "You should respond in JSON in this example:"

link_system_prompt += """
{
    "links": [
        {"type":"about page", "url":"https://www.latitudefinancial.com.au/about-us/"},
        {"type":""careers page, "url":"https://www.latitudefinancial.com.au/careers/"}
        ]
    }
"""

In [26]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About Page, or a Company page, or Careers/Jobs pages. 
You should respond in JSON in this example:
{
    "links": [
        {"type":"about page", "url":"https://www.latitudefinancial.com.au/about-us/"},
        {"type":""careers page, "url":"https://www.latitudefinancial.com.au/careers/"}
        ]
    }



In [29]:
## User prompt

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "Please decide which of these links are relevant to be included in the brochure of the \
    company, respond will the full https URL. Don't include Terms of Services, Privacy, email links. \n"

    user_prompt += "Links (some might be relative links): \n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [30]:
print(get_links_user_prompt(latitude))

Here is the list of links on the website of https://www.latitudefinancial.com.au/ - Please decide which of these links are relevant to be included in the brochure of the     company, respond will the full https URL. Don't include Terms of Services, Privacy, email links. 
Links (some might be relative links): 
/credit-cards/
/personal-loan
https://loans.portal.latitudefinancial.com.au/s/quote
https://www.latitudefinancial.com.au/credit-cards/28-degrees/
https://loans.portal.latitudefinancial.com.au/s/quote
https://www.latitudefinancial.com.au/credit-cards/28-degrees/
/personal-loan-repayment-calculator/
/budget-planner/
https://www.latitudefinancial.com.au/interest-free/find-a-store/
/car-loan/
/debt-consolidation-loans/
/home-renovation-loans/
/credit-cards/
https://latitudefs.zendesk.com/hc/en-au
https://www.creditsmart.org.au/
/hardship-care/
/digital-wallets/apple-pay/
/vulnerable-customers/


In [35]:
## Calling OpenAI 

def get_link(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system", "content":link_system_prompt},
            {"role":"user", "content": get_links_user_prompt(website)}
        ],
            response_format={"type": "json_object"}
    )

    result = response.choices[0].message.content
    return json.loads(result)

In [36]:
get_link("https://www.latitudefinancial.com.au/")

{'links': [{'type': 'about page',
   'url': 'https://www.latitudefinancial.com.au/about-us/'},
  {'type': 'careers page',
   'url': 'https://www.latitudefinancial.com.au/careers/'}]}

In [37]:
get_link("https://www.anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'events page', 'url': 'https://www.anthropic.com/events'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'}]}

## Make the Brochure

In [48]:
def get_all_the_details(url):
    result = "Landing page: \n"
    result += Website(url).get_contents()
    links = get_link(url)
    print("Found Links:" , links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [49]:
print(get_all_the_details("https://www.latitudefinancial.com.au/"))

Found Links: {'links': [{'type': 'about page', 'url': 'https://www.latitudefinancial.com.au/about-us/'}, {'type': 'careers page', 'url': 'https://www.latitudefinancial.com.au/careers/'}]}
Landing page: 
Webpage Title:
Credit Cards & Personal Loans Australia | Apply With Latitude
Webpage Contents:
Home
For life’s moments big and small
As your Partners in Money, our attitude is to be responsible, fast and always ready to help. So, what are you looking for today?
Credit Cards
Personal Loans
Check your Personal Loan interest rate in 2 mins.
With no impact to your credit score.
Get Rate Estimate
Get $200 bonus Latitude Rewards
When you spend $2000 on your Latitude 28º Mastercard every statement period in your first 3 months of approval.
Learn more
T&Cs, card fee and other charges apply. New and approved customers only. Offer ends 31 Dec 2024.
Check your Personal Loan interest rate in 2 mins.
With no impact to your credit score.
Get Rate Estimate
Get $200 bonus Latitude Rewards
When you spen

In [50]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [51]:
## 

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking for a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_the_details(url)
    user_prompt = user_prompt[:20000]
    return user_prompt

In [52]:
get_brochure_user_prompt("Latitude Financial Services", "https://www.latitudefinancial.com.au/")

Found Links: {'links': [{'type': 'about page', 'url': 'https://www.latitudefinancial.com.au/about-us/'}, {'type': 'careers page', 'url': 'https://www.latitudefinancial.com.au/careers/'}]}


"You are looking for a company called: Latitude Financial Services\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page: \nWebpage Title:\nCredit Cards & Personal Loans Australia | Apply With Latitude\nWebpage Contents:\nHome\nFor life’s moments big and small\nAs your Partners in Money, our attitude is to be responsible, fast and always ready to help. So, what are you looking for today?\nCredit Cards\nPersonal Loans\nCheck your Personal Loan interest rate in 2 mins.\nWith no impact to your credit score.\nGet Rate Estimate\nGet $200 bonus Latitude Rewards\nWhen you spend $2000 on your Latitude 28º Mastercard every statement period in your first 3 months of approval.\nLearn more\nT&Cs, card fee and other charges apply. New and approved customers only. Offer ends 31 Dec 2024.\nCheck your Personal Loan interest rate in 2 mins.\nWith no impact to your credit score.\nGet Rate Estimate\nGe

In [53]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system", "content":"system_prompt"},
            {"role":"user", "content":get_brochure_user_prompt(company_name, url)}
        ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [54]:
create_brochure("Latitude Financial Services", "https://www.latitudefinancial.com.au/")

Found Links: {'links': [{'type': 'about page', 'url': 'https://www.latitudefinancial.com.au/about-us/'}, {'type': 'careers page', 'url': 'https://www.latitudefinancial.com.au/careers/'}]}


```markdown
# Latitude Financial Services Brochure

**Webpage Title:** Credit Cards & Personal Loans Australia | Apply With Latitude

## About Us
Latitude Financial Services is a leading sales finance and consumer lending company that empowers people to shop and finance important moments in their lives. With over 2.1 million customers across Australia and New Zealand, we pride ourselves on being **Your Partners in Money**—offering responsible, fast, and customer-centered financial solutions.

### Our Story
Founded from a legacy of over a century in finance, Latitude has evolved through significant partnerships and acquisitions to become a trusted name in credit and loan solutions. Our roots go back to the 1920s and continue through our transition from GE to our current standing as a publically listed company on the Australian Stock Exchange.

## Our Products
We offer a variety of financial products tailored to meet your needs:

- **Credit Cards**: Flexible credit options with exclusive rewards.
- **Personal Loans**: Competitive rates and tailored repayment options.

### Special Offers
- **$200 Bonus Latitude Rewards**: Earn rewards when you spend $2000 on your Latitude 28º Mastercard during your first three months. *(T&Cs apply)*

## Why Choose Latitude?
- **Easy Application**: Quick and simple application process with a rapid response.
- **Transparent Pricing**: Clear disclosure of interest rates and fees—no hidden costs.
- **Personalized Service**: We assess applications individually and tailor offerings based on your circumstances.

## Tools & Resources
- **Calculators**: Use our personal loan repayment calculator and budget planner.
- **Help Centre**: Access FAQs, support for vulnerabilities, and more.
- **Digital Wallets**: Convenient payments with Apple Pay and Samsung Pay.

## Corporate Responsibility
At Latitude, we are committed to supporting our community and vulnerable customers. We believe in ethical lending and ensuring that our practices contribute positively to the lives of our customers.

## Career Opportunities
Join our passionate team driven by a commitment to better outcomes for people and the communities we serve. We value integrity, agility, and bold thinking.

### Life @ Latitude
At Latitude, you can be yourself, strive to be better, and help shape our future. *Explore our career opportunities today!*

## Connect with Us
For more information, visit our website or contact us for support.

- **Website**: [Latitude Financial Services](https://www.latitudefinancial.com)
- **Media Inquiries**: media.enquiries@latitudefinancial.com

---

Latitude Financial Services is here to be your go-to partner for all your financial needs, ensuring that your financial journey is as smooth and rewarding as possible. Whether you're looking to finance a new car, consolidate debts, or shop interest-free, Latitude is equipped to help you every step of the way.
```